In [3]:
import sympy as sp
from sympy import cos, sin
from sympy import init_printing
from IPython.display import Math, display
init_printing(use_latex=True)

def rotx_sym(th):
    R = sp.Matrix([[1, 0, 0],
                  [0, sp.cos(th), -sp.sin(th)],
                  [0, sp.sin(th), sp.cos(th)]])
    return R
def roty_sym(th):
    R = sp.Matrix([[sp.cos(th), 0, sp.sin(th)],
                   [0, 1, 0],
                   [-sp.sin(th), 0, sp.cos(th)]])
    return R
def rotz_sym(th):
    R = sp.Matrix([[sp.cos(th), -sp.sin(th), 0],
                   [sp.sin(th), sp.cos(th), 0],
                   [0, 0, 1]])
    return R

# making a symbolic SE3 function
def se3_sym(R = sp.eye(3), p = sp.Matrix([[0], [0], [0]])):

    T = sp.eye(4)
    T[0:3, 0:3] = R
    T[0:3, 3] = p
    
    return T

def get_A_sym(dh, jt_type = 'r', q=sp.Symbol('q')):
    if jt_type == 'r':
        A = se3_sym(R = rotz_sym(dh[0]+q)) @\
            se3_sym(p = sp.Matrix([[0], [0], [dh[1]]])) @\
            se3_sym(p = sp.Matrix([[dh[2]], [0], [0]])) @\
            se3_sym(R = rotx_sym(dh[3]))
    elif jt_type == 'p':
        A = se3_sym(R = rotz_sym(dh[0])) @\
            se3_sym(p = sp.Matrix([[0], [0], [dh[1]+q]])) @\
            se3_sym(p = sp.Matrix([[dh[2]], [0], [0]])) @\
            se3_sym(R = rotx_sym(dh[3]))
    else:
        A = None

    return A

In [15]:
# 2 (a) =======================================================================
q1, q2, d1, a2 = sp.symbols('q_1, q_2, d_1, a_2')

dh_param = [[0, d1, 0, sp.pi/2.0],
            [0, 0, a2, 0]]

T_01 = get_A_sym(dh_param[0], 'r', q1)
T_12 = get_A_sym(dh_param[1], 'r', q2)
T_02 = T_01 @ T_12
T = [T_01, T_02]

# Now get the Jacobian
# Axis of rotation and position of joint 1
z0 = sp.Matrix([[0],[0],[1]])
p0 = sp.Matrix([[0],[0],[0]])

# Axis of rotation and position of joint 2
z1 = T_01[:3, 2]
p1 = T_01[:3, 3]
# Position of end effector
p2 = T_02[:3, 3]

J = sp.zeros(6,2)

# Jv and Jw for joint 1
J[:3,0] = z0.cross(p2-p0)
J[3:,0] = z0

# Jv and Jw for joint 2
J[:3,1] = z1.cross(p2-p1)
J[3:,1] = z1


print("FINAL J:")
display(J)

FINAL J:


⎡-a₂⋅sin(q₁)⋅cos(q₂)             -a₂⋅sin(q₂)⋅cos(q₁)           ⎤
⎢                                                              ⎥
⎢a₂⋅cos(q₁)⋅cos(q₂)              -a₂⋅sin(q₁)⋅sin(q₂)           ⎥
⎢                                                              ⎥
⎢                           2                     2            ⎥
⎢         0           a₂⋅sin (q₁)⋅cos(q₂) + a₂⋅cos (q₁)⋅cos(q₂)⎥
⎢                                                              ⎥
⎢         0                            sin(q₁)                 ⎥
⎢                                                              ⎥
⎢         0                           -cos(q₁)                 ⎥
⎢                                                              ⎥
⎣         1                               0                    ⎦

In [16]:
# 2 (b) ===========================================================
import sys
sys.path.append("C:/Users/danie/Documents/School/MEEN537/HW")
sys.path.append("/home/daniel/Documents/MEEN537/HW/")
import kinematics as kin
from visualization import VizScene
import sympy as sp
import numpy as np
import time
np.set_printoptions(precision=4, suppress=True)

dh = [[0, 0.3, 0, np.pi/2.0],
      [0, 0, 0.3, 0]]

# An example of defining joint types which we may not have done yet. 
# The 4th joint, and 4th row of the DH parameters correspond to a prismatic joint. 
jt_types = ['r', 'r']

# notice how "jt_types" can be passed as an argument into "SerialArm"
arm = kin.SerialArm(dh, jt=jt_types)

# defining two different sets of joint angles
q_set1 = [0, 0]
q_set2 = [np.pi/4, np.pi/4]

# calculating two different jacobians for the two different joint configurations. 
J1 = arm.jacob(q_set1)
J2 = arm.jacob(q_set2)

print("from first set of q's, J is:")
print(J1)
print("from second set of q's, J is:")
print(J2)

from first set of q's, J is:
[[ 0.  -0. ]
 [ 0.3  0. ]
 [ 0.   0.3]
 [ 0.   0. ]
 [ 0.  -1. ]
 [ 1.   0. ]]
from second set of q's, J is:
[[-0.15   -0.15  ]
 [ 0.15   -0.15  ]
 [ 0.      0.2121]
 [ 0.      0.7071]
 [ 0.     -0.7071]
 [ 1.      0.    ]]


In [21]:
# 3 =======================================================================
q1, q2, a1, ac = sp.symbols('q_1, q_2, a_1, a_c')

dh_param = [[0, 0, a1, 0],
            [0, 0, ac, 0]]

T_01 = get_A_sym(dh_param[0], 'r', q1)
T_12 = get_A_sym(dh_param[1], 'r', q2)
T_02 = T_01 @ T_12

display(sp.trigsimp(T_02.evalf()))

# Now get the Jacobian
# Axis of rotation and position of joint 1
z0 = sp.Matrix([[0],[0],[1]])
p0 = sp.Matrix([[0],[0],[0]])

# Axis of rotation and position of joint 2
z1 = T_01[:3, 2]
p1 = T_01[:3, 3]
# Position of end effector
p2 = T_02[:3, 3]

J = sp.zeros(6,2)

# Jv and Jw for joint 1
J[:3,0] = z0.cross(p2-p0)
J[3:,0] = z0

# Jv and Jw for joint 2
J[:3,1] = z1.cross(p2-p1)
J[3:,1] = z1


print("FINAL J:")
display(sp.trigsimp(J.evalf()))

⎡cos(q₁)  -sin(q₁)  0  a₁⋅cos(q₁)⎤
⎢                                ⎥
⎢sin(q₁)  cos(q₁)   0  a₁⋅sin(q₁)⎥
⎢                                ⎥
⎢   0        0      1      0     ⎥
⎢                                ⎥
⎣   0        0      0      1     ⎦

⎡cos(q₂)  -sin(q₂)  0  a_c⋅cos(q₂)⎤
⎢                                 ⎥
⎢sin(q₂)  cos(q₂)   0  a_c⋅sin(q₂)⎥
⎢                                 ⎥
⎢   0        0      1       0     ⎥
⎢                                 ⎥
⎣   0        0      0       1     ⎦

⎡cos(q₁ + q₂)  -sin(q₁ + q₂)   0   a₁⋅cos(q₁) + a_c⋅cos(q₁ + q₂)⎤
⎢                                                               ⎥
⎢sin(q₁ + q₂)  cos(q₁ + q₂)    0   a₁⋅sin(q₁) + a_c⋅sin(q₁ + q₂)⎥
⎢                                                               ⎥
⎢     0              0        1.0                0              ⎥
⎢                                                               ⎥
⎣     0              0         0                1.0             ⎦

FINAL J:


⎡-a₁⋅sin(q₁) - a_c⋅sin(q₁ + q₂)  -a_c⋅sin(q₁ + q₂)⎤
⎢                                                 ⎥
⎢a₁⋅cos(q₁) + a_c⋅cos(q₁ + q₂)   a_c⋅cos(q₁ + q₂) ⎥
⎢                                                 ⎥
⎢              0                         0        ⎥
⎢                                                 ⎥
⎢              0                         0        ⎥
⎢                                                 ⎥
⎢              0                         0        ⎥
⎢                                                 ⎥
⎣             1.0                       1.0       ⎦

In [23]:
# 4 (b) ===========================================================
import sys
sys.path.append("C:/Users/danie/Documents/School/MEEN537/HW")
sys.path.append("/home/daniel/Documents/MEEN537/HW/")
import kinematics as kin
from visualization import VizScene
import sympy as sp
import numpy as np
import time
import transforms as tr
np.set_printoptions(precision=4, suppress=True)

dh = [[0, 0, 0, -np.pi/2.0],
      [0, 0.154, 0, np.pi/2.0],
      [0, 0.25, 0, 0],
      [-np.pi/2, 0, 0, -np.pi/2.0],
      [-np.pi/2, 0, 0, np.pi/2.0],
      [np.pi/2, 0.263, 0, 0]]
 
jt_types = ['r', 'r', 'p', 'r', 'r', 'r']

arm = kin.SerialArm(dh, tip=tr.se3(tr.roty(-np.pi/2)),jt=jt_types)

# calculating two different jacobians for the two different joint configurations. 
J_zeros = arm.jacob(q=[0,0,0,0,0,0])
J_offset = arm.jacob(q=[0,0,.1,0,0,0])

print(f"The Jacobian for q={[0,0,0,0,0,0]} is:\n{J_zeros}")
print(f"The Jacobian for q={[0,0,0.1,0,0,0]} is:\n{J_offset}")


# # Visualize the arm
# viz = VizScene()
# viz.add_arm(arm, draw_frames=True)
# viz.update(qs=[q])
# time_to_run = 30
# refresh_rate = 60

# for i in range(refresh_rate * time_to_run):
#     viz.update()
#     time.sleep(1.0/refresh_rate)

# viz.close_viz()


The Jacobian for q=[0, 0, 0, 0, 0, 0] is:
[[-0.417  0.25   0.    -0.263 -0.    -0.   ]
 [ 0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     1.     0.     0.263  0.   ]
 [ 0.     0.     0.     0.     1.     0.   ]
 [ 0.     1.     0.     0.     0.     1.   ]
 [ 1.     0.     0.     1.     0.     0.   ]]
The Jacobian for q=[0, 0, 0.1, 0, 0, 0] is:
[[-0.417  0.35   0.    -0.263 -0.    -0.   ]
 [ 0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     1.     0.     0.263  0.   ]
 [ 0.     0.     0.     0.     1.     0.   ]
 [ 0.     1.     0.     0.     0.     1.   ]
 [ 1.     0.     0.     1.     0.     0.   ]]
